In [2]:
import os
from langchain_groq import ChatGroq
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate
from dotenv import load_dotenv

In [3]:
load_dotenv() # Load environment variables from .env file

True

In [4]:
model = "llama3-8b-8192" # Model which is going to be used from Groq

In [5]:
llm = ChatGroq(model = model, temperature = 0.0) # Less creative, more factual responses
creative_llm = ChatGroq(model = model, temperature = 0.9) # More creative, less factual responses

In [6]:
# This is a sample article that we will use further.

article = """
Liverpool FC: The Renaissance Under Jürgen Klopp's Second Era

Liverpool Football Club is experiencing a remarkable transformation in what many are calling 
Jürgen Klopp's second era at the helm. After the departure of veteran players like Jordan 
Henderson, Roberto Firmino, and James Milner, the club has undergone a strategic rebuild 
that's breathing new life into the historic institution.

The midfield revolution has been particularly striking. The additions of Dominik Szoboszlai, 
Alexis Mac Allister, and Ryan Gravenberch have injected fresh energy into the team's core. 
These young talents, combined with the explosive potential of Darwin Núñez and the consistent 
brilliance of Mohamed Salah, have restored Liverpool's fearsome attacking reputation.

What's most impressive is how Klopp has managed this transition while maintaining the team's 
competitive edge. The Reds have seamlessly blended their renowned high-pressing style with 
a more nuanced possession game, adapting to the strengths of their new personnel. The 
emergence of academy graduates like Conor Bradley and Jarell Quansah also highlights the 
club's commitment to youth development.

However, challenges remain as the team seeks to reclaim their position at the summit of 
English football. The question of defensive stability and squad depth continues to spark 
debate among supporters. Yet, there's an undeniable sense of optimism at Anfield as this 
new-look Liverpool side shows all the hallmarks of another potentially dominant era in 
the club's storied history.
"""

In [7]:
# System prompt is a template that sets the context for the AI assistant.

system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant called {name} that helps generate article titles.", 
    input_variables = ["name"]
)

# User prompt is a template that asks (which is by the user) the AI assistant to generate a title based on the article content.
user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a name for a article.
The article is here for you to examine {article}

The name should be based of the context of the article.
Be creative, but make sure the names are clear, catchy,
and relevant to the theme of the article.

Only output the article name, no other explanation or
text can be provided.""",

    input_variables=["article"]
)

In [8]:
user_prompt.format(article = "TEST").content # This will print the formatted user prompt with the article content.

'You are tasked with creating a name for a article.\nThe article is here for you to examine TEST\n\nThe name should be based of the context of the article.\nBe creative, but make sure the names are clear, catchy,\nand relevant to the theme of the article.\n\nOnly output the article name, no other explanation or\ntext can be provided.'

In [9]:
first_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt]) # Chat prompt template combines system and user prompts.

In [10]:
print(first_prompt.format(article = "TEST", name = "GROQ")) # This will merge both the system and user prompts into a single prompt.

System: You are an AI assistant called GROQ that helps generate article titles.
Human: You are tasked with creating a name for a article.
The article is here for you to examine TEST

The name should be based of the context of the article.
Be creative, but make sure the names are clear, catchy,
and relevant to the theme of the article.

Only output the article name, no other explanation or
text can be provided.


In [11]:
# Using LangChain Expression Language (LCEL) to construct our chain. 
# The inputs are defined in dictionary {} and the pipe operator (|) and the contents of the left side are fed to right side of the pipe.

chain_one = (
    {
        "article" : lambda x: x["article"], 
        "name" : lambda x: x["name"]
    }
    | first_prompt
    | creative_llm
    | {"article_title": lambda x: x.content}
)

In [12]:
# We will use invoke method to execute the chain with the article and name as inputs.

article_title = chain_one.invoke({
    "article": article, 
    "name" : "GROQ"
})

article_title

{'article_title': '"Klopp\'s Revival: Liverpool\'s Rebirth Under the German Maestro\'s Second Era"'}

### Multiple LLMChains to follow!

In [13]:
# Here we change the system prompt and a new user prompt.

system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant that helps generate awesome articles."
)

second_user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a description for
the article. The article is here for you to examine:

---

{article}

---

Here is the article title '{article_title}'.

Output the SEO friendly article description. Also make sure dont exceed over 120 characters.
Do not output anything other than the description.""",
    input_variables=["article", "article_title"]
)

In [14]:
second_prompt = ChatPromptTemplate.from_messages(
    [
        system_prompt,
        second_user_prompt
    ]
)

In [15]:
chain_two = (
    {
        "article" : lambda x: x["article"],
        "article_title": lambda x: x["article_title"]
    }
    | second_prompt
    | llm
    | {"summary": lambda x: x.content}
)

# Here we have created a second chain.

In [16]:
article_desc_msg = chain_two.invoke(
    {
        "article": article,
        "article_title": article_title["article_title"]
    }
)

article_desc_msg   # This will invoke the second chain with the article and article title as inputs.
# The output will be the article description generated by the AI assistant.

{'summary': '"Jürgen Klopp\'s second era at Liverpool FC sparks a renaissance, as the Reds rebuild and adapt, blending youth and experience to reclaim their spot at the top of English football."'}

In [38]:
third_user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a new paragraph for the
article. The article is here for you to examine:

---

{article}

---

Choose one paragraph to review and edit. During your edit
ensure you provide constructive feedback to the user so they
can learn where to improve their own writing.""",
    input_variables=["article"]
)

# prompt template 3: creating a new paragraph for the article
third_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    third_user_prompt
])

In [39]:
from pydantic import BaseModel, Field

class Paragraph(BaseModel):
    original_paragraph: str = Field(description="The original paragraph")
    edited_paragraph: str = Field(description="The improved edited paragraph")
    feedback: str = Field(description=(
        "Constructive feedback on the original paragraph"
    ))

structured_llm = creative_llm.with_structured_output(Paragraph)

In [42]:
# chain 3: inputs: article / output: article_para
chain_three = (
    {"article": lambda x: x["article"]}
    | third_prompt
    | structured_llm
    | {
        "original_paragraph": lambda x: x.original_paragraph,
        "edited_paragraph": lambda x: x.edited_paragraph,
        "feedback": lambda x: x.feedback
    }
)

In [43]:
chain_three.invoke({"article": article})


{'original_paragraph': "The midfield revolution has been particularly striking. The additions of Dominik Szoboszlai, Alexis Mac Allister, and Ryan Gravenberch have injected fresh energy into the team's core. These young talents, combined with the explosive potential of Darwin Núñez and the consistent brilliance of Mohamed Salah, have restored Liverpool's fearsome attacking reputation.",
 'edited_paragraph': '',
 'feedback': "The paragraph does a great job of highlighting the changes made to the team's midfield, but could benefit from more specific examples of how these players have contributed to the team's success. Additionally, the transition between the mention of the current players and the team's attacking reputation could be smoother."}

In [50]:
# Remove the DALLE import
# from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from langchain_core.prompts import PromptTemplate

# Modified prompt template for generating image descriptions
image_prompt = PromptTemplate(
    input_variables=["article"],
    template=(
        "Generate a detailed image description in less than 100 words that captures "
        "the essence of this article. Focus on visual elements that would make a "
        "compelling illustration: {article}"
    )
)

# Modified display function to just print the image description
def display_image_prompt(image_description):
    print("\nImage Description Generated:")
    print("-" * 50)
    print(image_description)
    print("-" * 50)
    
# Wrap in RunnableLambda
image_display_runnable = RunnableLambda(display_image_prompt)

# Modified chain that outputs the image description
chain_four = (
    {"article": lambda x: x["article"]}
    | image_prompt
    | llm  # Using your existing Groq LLM
    | (lambda x: x.content)
    | image_display_runnable
)

In [51]:
chain_four.invoke({"article": article})


Image Description Generated:
--------------------------------------------------
Here is a detailed image description in under 100 words that captures the essence of the article:

The illustration depicts a vibrant, dynamic scene at Anfield Stadium, with Liverpool FC's iconic Liverbird logo emblazoned on the top left corner. In the foreground, a collage of players from the new-look squad is arranged in a circular pattern, showcasing the team's youthful energy and talent. Mohamed Salah, Darwin Núñez, and Dominik Szoboszlai are prominently featured, with the midfield trio of Alex Mac Allister, Ryan Gravenberch, and Conor Bradley forming the central axis. The background is a gradient of red and white, evoking the team's colors and the sense of optimism and renewal.
--------------------------------------------------
